In [1]:
import jutil
import numpy as np
from collections import Counter
import pyfpgrowth

In [2]:
sql = "select t.Course_code as course_code from train_plan_course t,train_plan_credit c where t.Course_big_type_id=c.sid \
and t.Grade='2014' and c.type_name like '%专业%'  and t.Speciality_code='0408'"
code = [d['course_code'] for d in jutil.load_data(sql)]

In [4]:
len(set(code))

53

In [3]:
sql = "select student_id,course_name,course_code,pmark,mark from view_stu_course_mark where speciality_code='0408' and grade='2014'"
df = jutil.load_pd_df(sql)
df.head()

,course_code,course_name,mark,pmark,student_id
0,0471005,面向对象程序设计实验,5,73.0,20122639
1,0471046,J2EE体系结构及程序设计,10,96.0,20122639
2,0471075,软件工程概论,9,87.0,20122639
3,0471076,人机交互技术,5,74.0,20122639
4,3200557,计算机导论A,8,82.0,20122639


In [4]:
# 选出了与专业相关的课程
dd = df[df['course_code'].isin(code)]

## 按学生group,然后将其课程分数作为维度,学生数作为数据量

In [118]:
code = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id'],as_index=False)['course_code'].apply(list)
mark = dd.drop_duplicates(subset=['student_id','course_code']).groupby(['student_id'],as_index=False)['mark'].apply(list)

In [119]:
d_mark = mark.to_dict()
d_code = code.to_dict()

In [127]:
transac = []
for k,cd in d_code.items():
    mk = d_mark.get(k,[])
    l = len(mk) if len(mk)<20 else 20
    arr = []
    for i in range(len(mk)):
#     for i in range(l):
        if mk[i]>'1':
            arr.append(cd[i]+'_0')
        else:
            arr.append(cd[i]+'_1')
#     arr = [cd[i]+'_'+mk[i] for i in range(len(mk))]
    transac.append(arr)

In [128]:
# 将数据写入文件
with open("/home/jimo/workspace/idea-community/FPGrowthTest/src/main/java/com/jimo/data.txt",'w') as f:
    for t in transac:
        line = ','.join(t)
        f.write(line)
        f.write('\n')

In [26]:
from pyspark.ml.fpm import FPGrowth
from pyspark.sql.functions import split

In [129]:
dddd = (spark.read.text("/home/jimo/workspace/idea-community/FPGrowthTest/src/main/java/com/jimo/data.txt").select(split("value", ",").alias("items")))
dddd.show(n=3,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|items                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               |
+-------------------------

In [131]:
fp = FPGrowth(minConfidence=0.7,minSupport=0.2,numPartitions=10)
fpm = fp.fit(dddd)
fpm.freqItemsets.show(5,truncate=False)

+-------------------------------------------------------+----+
|items                                                  |freq|
+-------------------------------------------------------+----+
|[9990620_0]                                            |110 |
|[9990620_0, 6011310_0]                                 |105 |
|[9990620_0, 6011310_0, 3243510_0]                      |103 |
|[9990620_0, 6011310_0, 3243510_0, 6023200_0]           |103 |
|[9990620_0, 6011310_0, 3243510_0, 6023200_0, 0474011_0]|103 |
+-------------------------------------------------------+----+
only showing top 5 rows



In [132]:
fpm.associationRules.show(5,truncate=False)

Py4JJavaError: An error occurred while calling o1278.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 248.0 failed 1 times, most recent failure: Lost task 0.0 in stage 248.0 (TID 361, localhost, executor driver): java.io.IOException: 设备上没有空间
	at java.io.FileOutputStream.writeBytes(Native Method)
	at java.io.FileOutputStream.write(FileOutputStream.java:326)
	at org.apache.spark.storage.TimeTrackingOutputStream.write(TimeTrackingOutputStream.java:58)
	at java.io.BufferedOutputStream.flushBuffer(BufferedOutputStream.java:82)
	at java.io.BufferedOutputStream.write(BufferedOutputStream.java:126)
	at net.jpountz.lz4.LZ4BlockOutputStream.flushBufferedData(LZ4BlockOutputStream.java:205)
	at net.jpountz.lz4.LZ4BlockOutputStream.write(LZ4BlockOutputStream.java:158)
	at java.io.ObjectOutputStream$BlockDataOutputStream.drain(ObjectOutputStream.java:1877)
	at java.io.ObjectOutputStream$BlockDataOutputStream.setBlockDataMode(ObjectOutputStream.java:178

In [107]:
from pymining import itemmining,assocrules

In [108]:
relim_input = itemmining.get_relim_input(transac)

In [109]:
item_sets = itemmining.relim(relim_input, min_support=5)

In [112]:
item_sets

{frozenset({'0471004_1'}): 5,
 frozenset({'0471076_1'}): 5,
 frozenset({'6011310_1'}): 5,
 frozenset({'6011320_1'}): 5,
 frozenset({'6111120_1'}): 5,
 frozenset({'3243700_0', '6111120_1'}): 5,
 frozenset({'0471005_1'}): 6,
 frozenset({'0471005_1', '0474020_0'}): 5,
 frozenset({'0471005_1', '0473041_0'}): 5,
 frozenset({'0471071_1'}): 7,
 frozenset({'0471071_1', '6033100_1'}): 5,
 frozenset({'3243781_1'}): 7,
 frozenset({'0471004_0', '3243781_1'}): 5,
 frozenset({'6100131_1'}): 8,
 frozenset({'3244152_1', '6100131_1'}): 5,
 frozenset({'0473031_1', '6100131_1'}): 6,
 frozenset({'3243510_0', '6100131_1'}): 5,
 frozenset({'3243700_0', '6100131_1'}): 5,
 frozenset({'0474012_0', '6100131_1'}): 6,
 frozenset({'0473041_0', '6100131_1'}): 5,
 frozenset({'0471004_0', '6100131_1'}): 5,
 frozenset({'0471075_1'}): 11,
 frozenset({'0471075_1', '6033100_1'}): 6,
 frozenset({'0471075_1', '3243020_1', '6033100_1'}): 5,
 frozenset({'0471075_1', '3243020_1'}): 5,
 frozenset({'0471075_1', '0473082_0'}): 5

In [110]:
ules = assocrules.mine_assoc_rules(item_sets, min_support=5, min_confidence=0.7)

In [111]:
ules

[(frozenset({'0471004_0',
             '0471005_0',
             '0471075_0',
             '0473031_0',
             '0473041_0',
             '0474019_0',
             '6111110_0'}),
  frozenset({'0471046_0'}),
  5,
  1.0),
 (frozenset({'0471004_0',
             '0471005_0',
             '0473031_0',
             '0473041_0',
             '0474019_0',
             '6111110_0'}),
  frozenset({'0471046_0', '0471075_0'}),
  5,
  1.0),
 (frozenset({'0471004_0', '0471005_0', '0473031_0', '0474019_0', '6111110_0'}),
  frozenset({'0471046_0', '0471075_0', '0473041_0'}),
  5,
  1.0),
 (frozenset({'0471004_0', '0471005_0', '0473031_0', '0474019_0'}),
  frozenset({'0471046_0', '0471075_0', '0473041_0', '6111110_0'}),
  5,
  0.7142857142857143),
 (frozenset({'0471005_0', '0473031_0', '0474019_0', '6111110_0'}),
  frozenset({'0471004_0', '0471046_0', '0471075_0', '0473041_0'}),
  5,
  0.7142857142857143),
 (frozenset({'0471004_0', '0473031_0', '0474019_0', '6111110_0'}),
  frozenset({'0471005_0',

In [19]:
transac

[['0471072_0',
  '0473082_0',
  '0474020_0',
  '3244238_0',
  '6011320_0',
  '6023200_0',
  '0471004_0',
  '6010500_0',
  '6041943_0',
  '0471054_0',
  '0473031_0',
  '3045600_0',
  '3243020_0',
  '6111110_0',
  '9990501_0',
  '0474003_0',
  '0474012_0',
  '0474014_0',
  '0474019_0',
  '3243781_0',
  '0471005_0',
  '0471046_0',
  '0471075_0',
  '0471076_0',
  '3200557_0',
  '3100312_0',
  '3200510_0',
  '3232400_0',
  '3243120_0',
  '3243510_0',
  '6033100_0',
  '9990620_0',
  '3143300_0',
  '3242400_0',
  '6011310_0',
  '6100131_0',
  '6111120_0',
  '0471024_0',
  '0471071_0',
  '0473041_0',
  '0474011_0',
  '3232100_0',
  '3243700_0',
  '3244152_0'],
 ['0471076_1',
  '0473041_0',
  '3200557_0',
  '3243120_0',
  '0474020_0',
  '3045600_0',
  '3100312_0',
  '3232100_1',
  '3243020_1',
  '3244238_0',
  '0471004_1',
  '0471046_0',
  '0474014_0',
  '3243510_0',
  '6011320_1',
  '6011320_1',
  '0471005_1',
  '0471071_1',
  '0474011_0',
  '3243700_0',
  '6010500_1',
  '0471071_1',
  '047401

In [16]:
p = pyfpgrowth.find_frequent_patterns(transactions=transac,support_threshold=5)

In [17]:
pyfpgrowth.generate_association_rules(p,0.7)

{('6111120_1',): (('0471076_0',), 1.0),
 ('6011310_1',): (('6010500_1',), 1.6666666666666667),
 ('0471004_1', '0474019_0'): (('3243781_0',), 0.8333333333333334),
 ('0471004_1', '3243700_0'): (('0474019_0', '3243781_0'), 1.0),
 ('0471004_1', '3243781_0'): (('0474019_0',), 0.8333333333333334),
 ('0471004_1', '0474019_0', '3243700_0'): (('3243781_0',), 1.0),
 ('0471004_1', '0474019_0', '3243781_0'): (('3243700_0',), 1.0),
 ('0471004_1', '3243700_0', '3243781_0'): (('0474019_0',), 1.0),
 ('0471004_1', '6033100_0'): (('3243781_0',), 1.0),
 ('0474012_0', '6100131_1'): (('0471075_1',), 0.8333333333333334),
 ('6033100_1', '6100131_1'): (('0474012_0',), 1.0),
 ('0471075_1', '6100131_1'): (('0471072_0',), 1.0),
 ('0471072_0', '6100131_1'): (('0471075_1',), 1.0),
 ('0471071_1', '0471072_0'): (('0471046_0', '3243510_0'), 0.7142857142857143),
 ('0471071_1', '3232100_1'): (('0471072_0',), 1.0),
 ('0471072_0', '3232100_1'): (('0473082_0',), 0.7142857142857143),
 ('0471071_1', '0471075_1'): (('0471072

In [6]:
transactions = [[1, 2, 5],
                [2, 4],
                [2, 3],
                [1, 2, 4],
                [1, 3],
                [2, 3],
                [1, 3],
                [1, 2, 3, 5],
                [1, 2, 3]]

p = pyfpgrowth.find_frequent_patterns(transactions=transactions,support_threshold=2)

In [8]:
pyfpgrowth.generate_association_rules(p,0.7)

{(1, 5): ((2,), 1.0),
 (2, 5): ((1,), 1.0),
 (4,): ((2,), 1.0),
 (5,): ((1, 2), 1.0)}